<a href="https://colab.research.google.com/github/konstantins7/rabota1/blob/main/vitrina__200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-auth requests pandas google-api-python-client beautifulsoup4 lxml

import requests
import pandas as pd
import datetime
import os
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession
from googleapiclient.discovery import build
from bs4 import BeautifulSoup
import logging

# Настройка логирования
logging.basicConfig(filename='/content/drive/MyDrive/Colab Notebooks/indexing_log.txt', level=logging.INFO)

# Константы
DAILY_QUOTA = 200  # Пример дневной квоты, замените на ваше значение

# Пути к файлам
VITRINA_KEY_FILE_LOCATION = '/content/drive/MyDrive/Colab Notebooks/vitrina24kz-82fd975fbbe4.json'
MED_VITRINA_KEY_FILE_LOCATION = '/content/drive/MyDrive/Colab Notebooks/medvitrina24kz-348e2b42e091.json'
VITRINA_TXT_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/vitrina24_urls.txt'
MED_VITRINA_TXT_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/medvitrina24_urls.txt'
VITRINA_PROGRESS_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/vitrina24_indexing_progress.csv'
MED_VITRINA_PROGRESS_FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/medvitrina24_indexing_progress.csv'
VITRINA_SITEMAP_URL = 'https://vitrina24.kz/sitemap.xml'
MED_VITRINA_SITEMAP_URL = 'https://med.vitrina24.kz/sitemap.xml'

SCOPES = ["https://www.googleapis.com/auth/indexing"]
INDEXING_URL = "https://indexing.googleapis.com/v3/urlNotifications:publish"

# Функция для логирования сообщений
def log_message(message):
    logging.info(f"{datetime.datetime.now()}: {message}")

# Функция для создания авторизованной сессии
def create_authed_session(key_file_location):
    credentials = service_account.Credentials.from_service_account_file(key_file_location, scopes=SCOPES)
    authed_session = AuthorizedSession(credentials)
    return authed_session, credentials

# Функция для проверки наличия файла
def check_file_exists(file_path):
    if not os.path.exists(file_path):
        log_message(f"Ошибка: Файл {file_path} не найден.")
        print(f"Ошибка: Файл {file_path} не найден.")
        return False
    return True

# Функция для загрузки URL-адресов из sitemap
def load_urls_from_sitemap(sitemap_url):
    response = requests.get(sitemap_url)
    soup = BeautifulSoup(response.content, 'xml')
    urls = [loc.text for loc in soup.find_all('loc')]
    return urls

# Функция для проверки квоты
def check_quota(credentials, domain_url):
    service = build('indexing', 'v3', credentials=credentials)
    try:
        response = service.urlNotifications().getMetadata(url=domain_url).execute()
        log_message(f"Информация о квоте: {response}")
        if 'latestUpdate' in response:
            return {'dailyLimitRemaining': DAILY_QUOTA - 1}  # Пример, замените на правильную логику проверки квоты
        else:
            return {'dailyLimitRemaining': DAILY_QUOTA}
    except Exception as e:
        log_message(f"Ошибка при проверке квоты: {e}")
        return None

# Функция для отправки URL-адреса на индексацию
def publish_url(authed_session, url):
    data = {
        "url": url,
        "type": "URL_UPDATED"
    }
    response = authed_session.post(INDEXING_URL, json=data)
    return response

# Функция для индексации URL-адресов
def index_urls(key_file_location, txt_file_path, progress_file_path, sitemap_url, domain_url):
    if not check_file_exists(txt_file_path):
        urls = load_urls_from_sitemap(sitemap_url)
        with open(txt_file_path, 'w') as file:
            for url in urls:
                file.write(url + '\n')
        log_message(f"Создан файл {txt_file_path} с URL из sitemap.")
    else:
        try:
            with open(txt_file_path, 'r') as file:
                urls = [line.strip() for line in file]
            log_message(f"Файл {txt_file_path} успешно прочитан.")
        except Exception as e:
            log_message(f"Ошибка при чтении файла {txt_file_path}: {e}")
            raise

    authed_session, credentials = create_authed_session(key_file_location)

    try:
        progress = pd.read_csv(progress_file_path)
        log_message(f"Файл прогресса {progress_file_path} успешно загружен.")
    except FileNotFoundError:
        progress = pd.DataFrame(urls, columns=['Product URL'])
        progress['Indexed'] = False
        progress.to_csv(progress_file_path, index=False)
        log_message(f"Создан новый файл прогресса: {progress_file_path}")

    today = datetime.date.today()
    urls_sent_today = 0

    quota_info = check_quota(credentials, domain_url)
    if quota_info and 'dailyLimitRemaining' in quota_info and quota_info['dailyLimitRemaining'] > 0:
        for index, row in progress.iterrows():
            if urls_sent_today >= DAILY_QUOTA:
                log_message("Достигнута дневная квота. Попробуйте снова завтра.")
                print("Достигнута дневная квота. Попробуйте снова завтра.")
                break

            if not row['Indexed']:
                url = row['Product URL']
                response = publish_url(authed_session, url)

                if response.status_code == 200:
                    progress.at[index, 'Indexed'] = True
                    urls_sent_today += 1
                    log_message(f"URL отправлен успешно: {url}")
                elif response.status_code == 429:
                    log_message("Достигнута дневная квота. Попробуйте снова завтра.")
                    print("Достигнута дневная квота. Попробуйте снова завтра.")
                    break
                elif response.status_code == 403:
                    log_message(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                    print(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                    break
                else:
                    log_message(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
                    print(f"Ошибка при отправке {url}: {response.status_code} {response.json()}")
    else:
        log_message("Достигнута дневная квота или произошла ошибка при проверке квоты.")
        print("Достигнута дневная квота или произошла ошибка при проверке квоты.")

    progress.to_csv(progress_file_path, index=False)
    log_message(f"Сегодня было отправлено {urls_sent_today} URL-адресов.")
    print(f"Сегодня было отправлено {urls_sent_today} URL-адресов.")

# Выполнение индексации для обоих сайтов
index_urls(VITRINA_KEY_FILE_LOCATION, VITRINA_TXT_FILE_PATH, VITRINA_PROGRESS_FILE_PATH, VITRINA_SITEMAP_URL, "https://vitrina24.kz")
index_urls(MED_VITRINA_KEY_FILE_LOCATION, MED_VITRINA_TXT_FILE_PATH, MED_VITRINA_PROGRESS_FILE_PATH, MED_VITRINA_SITEMAP_URL, "https://med.vitrina24.kz")
